In [9]:
import os
import subprocess
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import dicom2nifti
import dicom2nifti.exceptions as dcm_exceptions
import logging

# Set base directories
BASE_DIR = Path("./TCGA-SARC").resolve()
OUTPUT_DIR = BASE_DIR / "NIfTI"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Logging setup
log_file = BASE_DIR / "conversion_errors.log"
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

# Use `find` to locate all directories with DICOM files
print("[INFO] Scanning directories for DICOM files...")
find_command = f"find {BASE_DIR} -type f -iname '*.dcm' -printf '%h\n' | sort -u"
raw_output = subprocess.check_output(find_command, shell=True, text=True)
dicom_dirs = list(sorted(set(raw_output.strip().splitlines())))

print(f"[INFO] {len(dicom_dirs)} potential DICOM series found.\n")

# Prepare results
results = []

# Conversion loop
for dcm_dir in tqdm(dicom_dirs, desc="Converting"):
    folder = Path(dcm_dir)
    try:
        out_filename = OUTPUT_DIR / f"{folder.name}.nii.gz"
        dicom2nifti.convert_directory(dcm_dir, OUTPUT_DIR, reorient=True)
        results.append({"folder": str(folder), "status": "success", "output": str(out_filename)})
    except dcm_exceptions.ConversionValidationError as e:
        msg = str(e)
        logging.warning(f"[SKIPPED] {folder} - {msg}")
        results.append({"folder": str(folder), "status": f"skipped - {msg}"})
    except Exception as e:
        msg = str(e)
        logging.error(f"[ERROR] {folder} - {msg}")
        results.append({"folder": str(folder), "status": f"error - {msg}"})

# Save conversion summary
report_path = BASE_DIR / "conversion_report.csv"
pd.DataFrame(results).to_csv(report_path, index=False)

# Final stats
success_count = sum("success" in r["status"] for r in results)
failed_count = len(results) - success_count
print("\n[INFO] DICOM to NIfTI conversion completed.")
print(f"[INFO] Successful conversions: {success_count}")
print(f"[INFO] Failed/skipped conversions: {failed_count}")
print(f"[INFO] Log file: {log_file}")
print(f"[INFO] Summary CSV: {report_path}")


[INFO] Scanning directories for DICOM files...
[INFO] 33 potential DICOM series found.



Converting:   0%|          | 0/33 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/home/ricardoas/miniconda3/envs/radiomics/lib/python3.9/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ricardoas/miniconda3/envs/radiomics/lib/python3.9/site-packages/dicom2nifti/convert_dicom.py", line 118, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/ricardoas/miniconda3/envs/radiomics/lib/python3.9/site-packages/dicom2nifti/convert_generic.py", line 40, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Converting:   6%|▌         | 2/33 [00:03<00:48,  1.57s/it]Traceback (most recent call last):
  File "/home/ricardoas/miniconda3/envs/radiomics/lib/python3.9/site-packages/dicom2nifti/convert_dir.py", line 87, in co


[INFO] DICOM to NIfTI conversion completed.
[INFO] Successful conversions: 33
[INFO] Failed/skipped conversions: 0
[INFO] Log file: /home/ricardoas/Documents/GitHub/Deep-Learning-Based-Radiomics-for-Sarcoma-Prognosis/data/TCGA-SARC/conversion_errors.log
[INFO] Summary CSV: /home/ricardoas/Documents/GitHub/Deep-Learning-Based-Radiomics-for-Sarcoma-Prognosis/data/TCGA-SARC/conversion_report.csv
